# Neural Network Training - Water Levels

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim
torch.set_num_threads(4)
torch.set_num_interop_threads(4)

In [2]:
base_df = pd.read_csv('final_without_totals.csv')
base_df['aquifer_type_id'] = base_df['aquifer_type_id'].where(base_df['aquifer_type_id'] == 'confined', 0)
base_df['confined'] = base_df['aquifer_type_id'].where(base_df['aquifer_type_id'] == 0, 1)
base_df.drop(['aquifer_type_id', 'decadal_change', 'county_fips'], axis =1, inplace = True)
base_df = base_df.dropna()
base_df

,slope,well_depth,TP-TotPop_mean,TP-TotPop_slope,PS-WGWFr_mean,PS-WGWFr_slope,PS-WGWSa_mean,PS-WGWSa_slope,PS-WSWFr_mean,PS-WSWFr_slope,...,apr_slope_ET,may_slope_ET,jun_slope_ET,jul_slope_ET,aug_slope_ET,sep_slope_ET,oct_slope_ET,nov_slope_ET,dec_slope_ET,confined
0,0.698974,319.000000,63.37225,6.4143,5.7275,0.939,0.0,0.0,0.2500,-0.300,...,-0.010231,0.018840,0.079734,0.078744,0.100108,0.019087,0.035189,0.013260,0.005261,1
1,0.000000,110.000000,50.65425,-0.6701,8.1750,-1.162,0.0,0.0,0.0000,0.000,...,-0.007168,0.020043,0.050329,0.070892,0.046095,0.034398,0.042004,0.010613,0.007278,0
2,-0.070000,97.000000,14.17650,-0.2138,1.1850,-0.316,0.0,0.0,0.0000,0.000,...,-0.025363,0.010979,0.079713,0.072048,-0.006652,0.028606,0.026256,0.012102,0.006455,0
3,0.234000,120.000000,19.56875,-0.8005,4.2225,-1.223,0.0,0.0,0.0000,0.000,...,0.000335,0.084853,0.127095,0.058579,-0.031923,0.055627,0.039395,0.030780,0.017864,0
4,0.041000,160.000000,19.56875,-0.8005,4.2225,-1.223,0.0,0.0,0.0000,0.000,...,-0.016237,0.011932,0.026365,0.036504,0.055363,0.005465,0.034573,0.001122,0.004548,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21444,0.669231,465.000000,215.69150,7.2538,21.2975,-0.675,0.0,0.0,18.1300,-4.080,...,-0.023367,-0.063918,-0.060792,-0.059517,-0.049348,-0.045055,-0.024361,-0.010512,-0.003227,0
21445,1.017857,299.638938,895.57625,57.9219,131.3025,-11.795,0.0,0.0,140.5325,-21.743,...,-0.027429,-0.086441,-0.112566,-0.119920,-0.102341,-0.073428,-0.031203,-0.006618,-0.008993,0
21446,1.344156,120.000000,895.57625,57.9219,131.3025,-11.795,0.0,0.0,140.5325,-21.743,...,-0.025894,-0.084290,-0.111530,-0.119165,-0.101484,-0.072290,-0.031112,-0.006723,-0.008804,0
21447,2.342857,299.638938,244.12600,18.7802,38.3175,3.945,0.0,0.0,26.0125,-9.039,...,0.030956,0.022669,0.056658,0.012609,0.023476,0.040032,0.015884,0.008501,0.007605,0


In [3]:
#Train Test split
X = base_df.drop(['slope'], axis = 1)#.to_numpy()
y = np.array(base_df['slope'])
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.1, random_state=42)

In [4]:
scaler = StandardScaler()
data = X_train.drop(['confined'], axis = 1)
scaler.fit(data)

X_train_scaled = scaler.transform(X_train.drop(['confined'], axis = 1))
X_train = np.concatenate([X_train_scaled, np.array(list(X_train['confined'])).reshape(len(X_train), 1)], axis = 1)

X_valid_scaled = scaler.transform(X_valid.drop(['confined'], axis = 1))
X_valid = np.concatenate([X_valid_scaled, np.array(list(X_valid['confined'])).reshape(len(X_valid), 1)], axis = 1)

X_test_scaled = scaler.transform(X_test.drop(['confined'], axis =1))
X_test = np.concatenate([X_test_scaled, np.array(list(X_test['confined'])).reshape(len(X_test), 1)], axis = 1)

# set up the model and data structure

In [5]:
X_train_tensor = torch.as_tensor(X_train, dtype = torch.float)
y_train_tensor = torch.as_tensor(y_train, dtype = torch.float)
X_valid_tensor = torch.as_tensor(X_valid, dtype = torch.float)
y_valid_tensor = torch.as_tensor(y_valid, dtype = torch.float)
X_test_tensor = torch.as_tensor(X_test, dtype = torch.float)
y_test_tensor = torch.as_tensor(y_test, dtype = torch.float)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [6]:
class NN(nn.Module):
    def __init__(self, hidden1, hidden2, hidden3, hidden4, drop):
        super(NN, self).__init__()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=len(base_df.columns)-1, out_features=hidden1),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(in_features=hidden1, out_features=hidden2),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(in_features=hidden2, out_features=hidden3),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(in_features=hidden3, out_features=hidden4),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(in_features=hidden4, out_features=1)
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

In [7]:
#train loop for training MLP through gradient descent
def train_loop(dataloader, model, loss_fn, optimizer, device):

    model.train() # Set your model to "train" mode

    for batch_idx, (X, y) in enumerate(dataloader):
        # Send inputs and labels to device
        y=y.reshape(-1)
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss. reshape predictions
        pred = model(X).reshape(-1) 

        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

In [8]:
#If evaluating model training while it trains, include function below with train loop
def test_loop(dataloader, model, loss_fn, device, count): 
    total = len(dataloader.dataset)
    test_loss = 0

    model.eval() # Set model to "eval" mode

    with torch.no_grad():
        for X, y in dataloader:
            # Send inputs and labels to device
            X, y = X.to(device), y.to(device)

            pred = model(X).reshape(-1) # Reshape the pred of shape (N, 1) to (N,)
            loss = loss_fn(pred, y)
            #no optimization 

            test_loss += loss.item() * X.size(0) # because "loss" was averaged over the batch

    test_loss /= total
    
    #uncomment below to track test loss
    if count % 30 == 0:
        print(f"Avg test loss: {test_loss:>8f} \n")


    return test_loss

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
count = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
losses = [(nn.functional.mse_loss, 'mse')]


#hyperparameters. Edit these as needed
dropouts = [.15]
lrs = [5e-3, 5e-2, 1e-1]
epoch_size = [300, 450]
batch_size = [100, 300, 500,700]
hidden_sizes = [[150,100, 50, 25], [200,100, 50, 25], [150,100,50, 10], [200, 100,20,5]]


error_vals = []
lr_vals = []
batches = []
epochs = []
loss_functions = []
drop_vals = []
hidden_vals = []

for loss_fn in losses:
    for hiddens in hidden_sizes:
        for ep in epoch_size:
            for bs in batch_size:
                for drop in dropouts:
                    for lr_ in lrs:
                        model = NN(hiddens[0], hiddens[1], hiddens[2], hiddens[3], drop).to(device)
                        train_dataloader = DataLoader(train_dataset, batch_size = bs, shuffle = True)
                        valid_dataloader = DataLoader(valid_dataset, batch_size = bs, shuffle = True)
                        test_dataloader = DataLoader(test_dataset, batch_size = bs, shuffle = False)
                        optimizer = optim.SGD(params = model.parameters(), lr = lr_)
                        
                        #keep track as model training runs
                        display('h1: ' + str(hiddens[0]) + ' - h2: ' + str(hiddens[1]) + ' - h3: ' + str(hiddens[2]) + ' - h4: ' + str(hiddens[3]) + ' -- lr = ' + str(lr_) + ' -- ep = ' + str(ep) + ' -- bs = ' + str(bs))
                        
                        #train the model
                        for t in range(ep):
                            train_loop(train_dataloader, model, loss_fn[0], optimizer, device)
                            test_loop(valid_dataloader, model, loss_fn[0], device, count)
                            count +=1
        
                        #make predictions and pull truth values
                        final_loss = test_loop(test_dataloader, model, loss_fn[0], device, count)
                        display('final loss: ' + str(final_loss))
                        
                        #Add values to lists, which will then populate the hyper-parameter and evaluate metric dataframe
                        error_vals.append(final_loss)
                        lr_vals.append(lr_)
                        batches.append(bs)
                        epochs.append(ep)
                        loss_functions.append(loss_fn[1])
                        drop_vals.append(drop)
                        hidden_vals.append('h1: ' + str(hiddens[0]) + ' - h2: ' + str(hiddens[1]) + ' - h3: ' + str(hiddens[2]) + ' - h4: ' + str(hiddens[3]))

#Output Dataframe
output_df = pd.DataFrame({'hiddens': hidden_vals, 'batch size': batches, 'epochs':epochs, 
                          'lr': lr_vals, 'loss func':loss_functions , 'dropout': drop_vals, 'test_mse':error_vals})
output_df['data in'] = 'w/out totals, w aqtype, st scaler, slope'
